In [1]:
import os
import zipfile

# 定义拆分文件所在的目录
split_files_dir = "/kaggle/input/pcbdata"

# 定义合并后文件的路径
merged_file_path = "/kaggle/working/datasets.zip"

# 定义解压目标目录
extract_dir = "/kaggle"

# 获取拆分文件列表并按文件名排序
split_files = sorted([os.path.join(split_files_dir, f) for f in os.listdir(split_files_dir) if f.startswith('datasets.zip.part')])

try:
    # 以二进制写入模式打开合并后的文件
    with open(merged_file_path, 'wb') as outfile:
        for split_file in split_files:
            # 以二进制读取模式打开每个拆分文件
            with open(split_file, 'rb') as infile:
                # 将拆分文件的内容写入合并后的文件
                outfile.write(infile.read())
    print(f"文件已成功合并到 {merged_file_path}")

    # 检查合并后的文件是否存在
    if os.path.exists(merged_file_path):
        # 创建解压目标目录（如果不存在）
        if not os.path.exists(extract_dir):
            os.makedirs(extract_dir)

        # 以读取模式打开合并后的 zip 文件
        with zipfile.ZipFile(merged_file_path, 'r') as zip_ref:
            # 解压文件到指定目录
            zip_ref.extractall(extract_dir)
        print(f"{merged_file_path} 已成功解压到 {extract_dir}")
    else:
        print(f"合并后的文件 {merged_file_path} 不存在，无法进行解压操作。")
except Exception as e:
    print(f"操作过程中出现错误: {e}")

文件已成功合并到 /kaggle/working/datasets.zip
/kaggle/working/datasets.zip 已成功解压到 /kaggle


In [2]:
### 创建目标目录
!mkdir -p /kaggle/working/ultralytics

# 递归复制整个目录（保留层级结构）
!cp -r /kaggle/input/yolov8-ecoal-attn/ultralytics-main/ultralytics/* /kaggle/working/ultralytics/

In [3]:
import sys
import torch
from ultralytics import YOLO

# 设置sys.path 以加载自定义的Ultralytics代码
sys.path.insert(0, '/kaggle/working/ultralytics')

# 启用异常检测，方便定位梯度/forward问题
torch.autograd.set_detect_anomaly(True)

### 1. 初始化自定义模型（修改了yaml+loss+trainer）
model = YOLO('/kaggle/working/ultralytics/cfg/models/v8/yolov8-ECoAl.yaml')

### 2. 设置训练参数
results = model.train(
    # 数据集路径
    data='/kaggle/working/ultralytics/cfg/datasets/PCB.yaml',
    epochs=200,
    batch=8,
    lr0=1e-3,  # 极低学习率
    optimizer='AdamW',
    weight_decay=0,
    cos_lr=True,
    warmup_epochs=10,
    box=5.0,  # 适度降低定位权重
    cls=1.0,  # 提高分类权重
    kobj=2.0,  # 新增目标存在性损失权重

    amp=False, 

)

### 3. 训练完成后，查看训练结果
# 训练完成后，查看训练结果文件
!ls /kaggle/working/runs/detect/

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
New https://pypi.org/project/ultralytics/8.3.176 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.83 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/ultralytics/cfg/models/v8/yolov8-ECoAl.yaml, data=/kaggle/working/ultralytics/cfg/datasets/PCB.yaml, epochs=200, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, pro

100%|██████████| 755k/755k [00:00<00:00, 16.9MB/s]
E0000 00:00:1754704507.588569      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754704507.697068      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7522  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  1     32580  ultralytics.nn.modules.block.C2fECoA         [64, 64, 1]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  1    128964  ultralytics.nn.modules.block.C2fECoA         [128, 128, 1]                 
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /kaggle/datasets/labels/train2017... 593 images, 0 backgrounds, 0 corrupt: 100%|██████████| 593/593 [00:00<00:00, 787.58it/s]

train: New cache created: /kaggle/datasets/labels/train2017.cache


...


200 epochs completed in 2.150 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.83 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8-ECoAl summary (fused): 162 layers, 2,677,508 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.27it/s]


                   all        100        406      0.845      0.829      0.868      0.453
          missing_hole          9         37      0.632          1      0.995      0.655
            mouse_bite         17         69      0.929      0.783      0.824        0.4
          open_circuit         11         43      0.913      0.731      0.803       0.38
                 short         25         98       0.83      0.959      0.946      0.481
                  spur         23         98        0.9      0.765      0.819      0.394
       spurious_copper         15         61      0.865      0.735      0.822       0.41


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.3ms preprocess, 7.0ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train
train
